In [1]:
from task2.models.base import Base, engine

# Create all tables in the database
Base.metadata.create_all(engine)
print("Database tables created successfully!")

2025-04-13 00:25:32,027 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-04-13 00:25:32,027 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("departments")
2025-04-13 00:25:32,028 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-04-13 00:25:32,028 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("employees")
2025-04-13 00:25:32,028 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-04-13 00:25:32,029 INFO sqlalchemy.engine.Engine COMMIT
Database tables created successfully!
